In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [2]:
#to get 10 page
BASE_URL = "http://ufcstats.com"
MATCH_URL = f"{BASE_URL}/statistics/events/completed?page="

In [29]:
match_links = [] #all match link in 10 page

for page in range(1, 19):
    url = f"{MATCH_URL}{page}" #get all url till page 10 and jan
    print(f"Scraping event list page {page}: {url}") #print page link
    
    response = requests.get(url) #req again to every page

    soup = BeautifulSoup(response.text, "html.parser") #get html tag in all page
    
    rows = soup.find_all("tr", class_="b-statistics__table-row") # in html find links of match in each page
    
    for row in rows: # in every row
        link_tag = row.find("a") #find match links tag
        date_tag = row.find("span", class_="b-statistics__date") #find date
        
        if link_tag and date_tag:
            date_text = date_tag.text.strip()
            if date_text == "January 03, 2015":
                match_links.append(link_tag["href"])
                print("Reached Jan 03, 2015 — stopping.") #stop at 2015
                break
            match_links.append(link_tag["href"]) #save match link #all match link in 10 page. there are 245 match
    else:
        continue
    break  # stop outer loop too once we reach Jan 03, 2015

print(f"Collected {len(match_links)} match links.")
match_links[:5] #print first 5 match links


Scraping event list page 1: http://ufcstats.com/statistics/events/completed?page=1
Scraping event list page 2: http://ufcstats.com/statistics/events/completed?page=2
Scraping event list page 3: http://ufcstats.com/statistics/events/completed?page=3
Scraping event list page 4: http://ufcstats.com/statistics/events/completed?page=4
Scraping event list page 5: http://ufcstats.com/statistics/events/completed?page=5
Scraping event list page 6: http://ufcstats.com/statistics/events/completed?page=6
Scraping event list page 7: http://ufcstats.com/statistics/events/completed?page=7
Scraping event list page 8: http://ufcstats.com/statistics/events/completed?page=8
Scraping event list page 9: http://ufcstats.com/statistics/events/completed?page=9
Scraping event list page 10: http://ufcstats.com/statistics/events/completed?page=10
Scraping event list page 11: http://ufcstats.com/statistics/events/completed?page=11
Scraping event list page 12: http://ufcstats.com/statistics/events/completed?page=1

['http://ufcstats.com/event-details/0e2c2daf11b5d8f2',
 'http://ufcstats.com/event-details/7956f026e2672c47',
 'http://ufcstats.com/event-details/38e5d9dcb0fddc42',
 'http://ufcstats.com/event-details/9336e86cfd4ceaa1',
 'http://ufcstats.com/event-details/8944a0f9b2f0ce6d']

In [30]:
from tqdm import tqdm

fight_row =[]
stats_link =[]

for match in tqdm(match_links, desc="scarpe fight"):
    response = requests.get(match)
    soup = BeautifulSoup(response.text, "html.parser")

    #in each match link table
    #find all right row
    fight_rows = soup.find_all('tr', class_="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click")
    fight_row.append(fight_rows)

    for row in tqdm(fight_rows, desc="filter category"):
        tds = row.find_all("td") #find all td in each fight row
        if len(tds) >= 9: #total row is 9 - 0~9
            #get data in each row
            #get weight in 6 row. find the p tag with info
            weight_class = tds[6].find("p").get_text(strip=True)
            #get method in 7 row. have 2 p tag get the first one
            method = tds[7].find_all("p")[0].get_text(strip=True)
            #get total round in 8 row
            total_round = tds[8].find("p").get_text(strip=True) #1,2,3 - total round that ends

            #get only matching category
            category = weight_class == "Lightweight" and (method in ["KO/TKO", "KO", "TKO"])
            if category:
                #print("match found",weight_class, method)
                #get data link to go to the stats table and extract data 
                data_link = row.get("data-link") # all fight link that matched the category
                #print(data_link)
                stats_link.append(data_link)

print(f"Collected {len(stats_link)} stats links.")
stats_link[:5] #print first 5 stats links



scarpe fight: 100%|██████████| 449/449 [07:30<00:00,  1.00s/it]

Collected 257 stats links.


['http://ufcstats.com/fight-details/cb902d73389ed34b',
 'http://ufcstats.com/fight-details/ecdbb3a5216b815e',
 'http://ufcstats.com/fight-details/7cbfeba85f86d1bf',
 'http://ufcstats.com/fight-details/202c47db69768356',
 'http://ufcstats.com/fight-details/69b31593175f77ec']

In [ ]:
stats_link

In [ ]:
#go in each filtered stats_link get the data
# req again with the fight link to go to stats table
for all_stats in stats_link:
    response = requests.get(all_stats)
    soup = BeautifulSoup(response.text, "html.parser")

    # extract data in stats table page for top div 
    fight_details = soup.find("div", class_="b-fight-details__fight")
    if fight_details:
        # get round num
        total_round = fight_details.find("i", class_="b-fight-details__text-item").get_text(strip=True)
        # get method ko/tko
        method_tag = fight_details.find("i", attrs={"style": "font-style: normal"})
        method = method_tag.get_text(strip=True) if method_tag else "-"

    # extract data from table
    table_stats = soup.find_all("table", class_="b-fight-details__table js-fight-table")
    #print(len(table_stats)) #2
    table1= table_stats[0] #top totals table third table
    table2= table_stats[1] #significant table body attack forth table
    #extract data for one match from 2 table. 
    #now need to do this for all match 

    #save data
    kd_list, sig_str_list, sig_str_pct_list, total_str_list = [], [], [], []
    td_attempt_list, td_pct_list, sub_att_list, rev_list, ctrl_sec_list = [], [], [], [], []
    head_list, body_list, leg_list, distance_list, clinch_list, ground_list = [], [], [], [], [], []


    if table1:
        t1_row = table1.find_all("tr", class_="b-fight-details__table-row")
        for section in t1_row:
            #round_num = section.find("tr", class_="b-fight-details__table-col").get_text(strip=True) # get round num tr
            td_stats = section.find_all("td")
            if len(td_stats) >= 10:  # total col is 10
                # get data in each row ex: round 1
                kd = [p.get_text(strip=True) for p in td_stats[1].find_all("p")]
                sig_str = [p.get_text(strip=True) for p in td_stats[2].find_all("p")]
                sig_str_pct = [p.get_text(strip=True) for p in td_stats[3].find_all("p")]
                total_str = [p.get_text(strip=True) for p in td_stats[4].find_all("p")]
                td_attempt = [p.get_text(strip=True) for p in td_stats[5].find_all("p")]
                td_pct = [p.get_text(strip=True) for p in td_stats[6].find_all("p")]
                sub_att = [p.get_text(strip=True) for p in td_stats[7].find_all("p")]
                rev = [p.get_text(strip=True) for p in td_stats[8].find_all("p")]
                ctrl_sec = [p.get_text(strip=True) for p in td_stats[9].find_all("p")]
                # each list above has [fighter1, fighter2]
                # append each row data
                kd_list.append(kd)
                sig_str_list.append(sig_str)
                sig_str_pct_list.append(sig_str_pct)
                total_str_list.append(total_str)
                td_attempt_list.append(td_attempt)
                td_pct_list.append(td_pct)
                sub_att_list.append(sub_att)
                rev_list.append(rev)
                ctrl_sec_list.append(ctrl_sec)

    if table2:
        t2_row = table2.find_all("tr", class_="b-fight-details__table-row")               
        for section2 in t2_row:
            #round_num2 = section2.find("tr", class_="b-fight-details__table-col").get_text(strip=True) # get round num tr
            td_stats_2 = section2.find_all("td")
            if len(td_stats_2) >= 8:
                #get body attack data
                head = [p.get_text(strip=True) for p in td_stats_2[3].find_all("p")]
                body = [p.get_text(strip=True) for p in td_stats_2[4].find_all("p")]
                leg = [p.get_text(strip=True) for p in td_stats_2[5].find_all("p")]
                distance = [p.get_text(strip=True) for p in td_stats_2[6].find_all("p")]
                clinch = [p.get_text(strip=True) for p in td_stats_2[7].find_all("p")]
                ground = [p.get_text(strip=True) for p in td_stats_2[8].find_all("p")]

                # append each row data
                head_list.append(head)
                body_list.append(body)
                leg_list.append(leg)
                distance_list.append(distance)
                clinch_list.append(clinch)
                ground_list.append(ground)

    print(
        "new fight-",
        "total_round:", total_round,
        "KD:", kd_list,
        "Sig.Str (L/A):", sig_str_list,
        "sig_str_pct:", sig_str_pct_list,
        "total_str (L/A):", total_str_list,
        "td (L/A):", td_attempt_list,
        "td_pct:", td_pct_list,
        "sub_att:", sub_att_list,
        "rev:", rev_list,
        "Control(sec):", ctrl_sec_list,
        "head (L/A):", head_list,
        "body (L/A):", body_list,
        "leg (L/A):", leg_list,
        "distance (L/A):", distance_list,
        "clinch (L/A):", clinch_list,
        "ground (L/A):", ground_list,
        )

In [36]:
 # === CLEANING USING REGEX ===
import re

def split_of_list(lst):
                                    """Split ['22 of 41', '23 of 64'] → [[22, 41], [23, 64]]"""
                                    clean = []
                                    for val in lst:
                                        match = re.findall(r'(\d+)\s*of\s*(\d+)', val)
                                        if match:
                                            clean.append([int(match[0][0]), int(match[0][1])])
                                        else:
                                            clean.append([0, 0])
                                    return clean

def clean_pct_list(lst):
                                    """Remove '%' and convert to float"""
                                    return [float(re.sub(r'[^0-9.]', '', v)) if re.search(r'\d', v) else 0.0 for v in lst]

def clean_int_list(lst):
                                    """Convert list of strings to int"""
                                    return [int(re.sub(r'\D', '', v)) if re.search(r'\d', v) else 0 for v in lst]

def ctrl_to_seconds(lst):
                                    """Convert ['1:23', '0:45'] → [83, 45]"""
                                    result = []
                                    for v in lst:
                                        match = re.findall(r'(\d+):(\d+)', v)
                                        if match:
                                            m, s = map(int, match[0])
                                            result.append(m * 60 + s)
                                        else:
                                            result.append(0)
                                    return result

In [45]:
from itertools import zip_longest  # add if you want safer pairing (optional)
import re
import csv 
from collections import defaultdict

fight_data = defaultdict(list)
# go in each filtered stats_link get the data
for all_stats in stats_link:
    response = requests.get(all_stats)
    soup = BeautifulSoup(response.text, "html.parser")

    # extract data in stats table page for top div 
    fight_details = soup.find("div", class_="b-fight-details__fight")
    if fight_details:
        # get round num (total rounds)
        total_round = fight_details.find("i", class_="b-fight-details__text-item").get_text(strip=True)
        # get method ko/tko
        method_tag = fight_details.find("i", attrs={"style": "font-style: normal"})
        method = method_tag.get_text(strip=True) if method_tag else "-"

    # extract data from table
    table_stats = soup.find_all("table", class_="b-fight-details__table js-fight-table")
    if len(table_stats) < 2:
        continue

    table1 = table_stats[0]  # top totals table
    table2 = table_stats[1]  # significant strikes table

    # get all rows first
    all_t1_rows = table1.find_all("tr", class_="b-fight-details__table-row")
    all_t2_rows = table2.find_all("tr", class_="b-fight-details__table-row")

    # FILTER rows to only keep "data" rows (avoid header/separator rows)
    # table1 data rows must have at least 10 <td>, table2 data rows at least 9 <td>
    t1_rows = [r for r in all_t1_rows if len(r.find_all("td")) >= 10]
    t2_rows = [r for r in all_t2_rows if len(r.find_all("td")) >= 9]

    # optional debug prints to confirm alignment (uncomment if needed)
    # print("debug:", all_stats, "t1_rows:", len(t1_rows), "t2_rows:", len(t2_rows))

    # iterate data rows and pair by index
    for idx, section in enumerate(t1_rows, start=1):
        td_stats = section.find_all("td")
        # round number is idx
        round_num = idx

        kd = [p.get_text(strip=True) for p in td_stats[1].find_all("p")]
        sig_str = [p.get_text(strip=True) for p in td_stats[2].find_all("p")]
        sig_str_pct = [p.get_text(strip=True) for p in td_stats[3].find_all("p")]
        total_str = [p.get_text(strip=True) for p in td_stats[4].find_all("p")]
        td_attempt = [p.get_text(strip=True) for p in td_stats[5].find_all("p")]
        td_pct = [p.get_text(strip=True) for p in td_stats[6].find_all("p")]
        sub_att = [p.get_text(strip=True) for p in td_stats[7].find_all("p")]
        rev = [p.get_text(strip=True) for p in td_stats[8].find_all("p")]
        ctrl_sec = [p.get_text(strip=True) for p in td_stats[9].find_all("p")]

        # get matching table2 row if exists (safe access)
        if round_num - 1 < len(t2_rows):
            section2 = t2_rows[round_num - 1]
            td_stats_2 = section2.find_all("td")
            if len(td_stats_2) >= 9:
                head = [p.get_text(strip=True) for p in td_stats_2[3].find_all("p")]
                body = [p.get_text(strip=True) for p in td_stats_2[4].find_all("p")]
                leg = [p.get_text(strip=True) for p in td_stats_2[5].find_all("p")]
                distance = [p.get_text(strip=True) for p in td_stats_2[6].find_all("p")]
                clinch = [p.get_text(strip=True) for p in td_stats_2[7].find_all("p")]
                ground = [p.get_text(strip=True) for p in td_stats_2[8].find_all("p")]
            else:
                head = body = leg = distance = clinch = ground = ["-", "-"]
        else:
            head = body = leg = distance = clinch = ground = ["-", "-"]

        sig_str = split_of_list(sig_str)
        total_str = split_of_list(total_str)
        td_attempt = split_of_list(td_attempt)
        sig_str_pct = clean_pct_list(sig_str_pct)
        td_pct = clean_pct_list(td_pct)
        sub_att = clean_int_list(sub_att)
        rev = clean_int_list(rev)
        kd = clean_int_list(kd)
        ctrl_sec = ctrl_to_seconds(ctrl_sec)
        head = split_of_list(head)
        body = split_of_list(body)
        leg = split_of_list(leg)
        distance = split_of_list(distance)
        clinch = split_of_list(clinch)
        ground = split_of_list(ground)
        total_round_c = total_round.replace("Round:", "").strip()

       # store the round data in dictionary
        round_data = {
            "link": all_stats,  # use the fight link as group key
            "round": round_num,
            "kd": kd,
            "sig_str": sig_str,
            "sig_str_pct": sig_str_pct,
            "total_str": total_str,
            "td_attempt": td_attempt,
            "td_pct": td_pct,
            "sub_att": sub_att,
            "rev": rev,
            "ctrl_sec": ctrl_sec,
            "head": head,
            "body": body,
            "leg": leg,
            "distance": distance,
            "clinch": clinch,
            "ground": ground,
            "total_round": total_round_c,
            "method": method
        }

        # group all rounds that belong to the same fight link
        fight_data[all_stats].append(round_data)
        time.sleep(0.5)

fight_counter = 1
fights_by_round_count = defaultdict(list)

# Define your preferred column order
columns = [
    "fight",
    "round",
    "kd",
    "sig_str",
    "sig_str_pct",
    "total_str",
    "td_attempt",
    "td_pct",
    "sub_att",
    "rev",
    "ctrl_sec",
    "head",
    "body",
    "leg",
    "distance",
    "clinch",
    "ground",
    "total_round",
    "method"
]

for link, rounds in fight_data.items():
    # total_round is same for the fight (use from first round)
    total_round_c = rounds[0]["total_round"]
    fight_name = f"fight{fight_counter}"

    for r in rounds:
        row = {"fight": fight_name, "round": r["round"]}
        for k, v in r.items():
            if k not in ["link", "round", "total_round"]:
                row[k] = v
        fights_by_round_count[total_round_c].append(row)

    fight_counter += 1

# save grouped by total rounds
for total_round_c, rows in fights_by_round_count.items():

    if total_round_c in ["4", "5"]:
        continue

    filename = f"ufc_totalround_{total_round_c}.csv"
    fieldnames = sorted(rows[0].keys())
    

    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=columns)
        writer.writeheader()
        writer.writerows(rows)

    print(f"✅ Saved {filename} ({len(rows)} rows)")
    


✅ Saved ufc_totalround_1.csv (135 rows)
✅ Saved ufc_totalround_3.csv (102 rows)
✅ Saved ufc_totalround_2.csv (166 rows)


In [ ]:
fight_counter = 1
fights_by_round_count = defaultdict(list)

for link, rounds in fight_data.items():
    # total_round is same for the fight (use from first round)
    total_round = rounds[0]["total_round"]
    fight_name = f"fight{fight_counter}"

    for r in rounds:
        row = {"fight": fight_name, "round": r["round"]}
        for k, v in r.items():
            if k not in ["link", "round", "total_round"]:
                row[k] = v
        fights_by_round_count[total_round].append(row)

    fight_counter += 1

# save grouped by total rounds
for total_round, rows in fights_by_round_count.items():
    filename = f"ufc_totalround_{total_round}.csv"
    fieldnames = sorted(rows[0].keys())

    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

    print(f"✅ Saved {filename} ({len(rows)} rows)")


In [8]:
import csv

def save_to_csv(filename, data_list):
    """Save list of dicts to CSV if not empty."""
    if not data_list:
        print(f"⚠️ No data to save for {filename}")
        return
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=list(data_list[0].keys()))
        writer.writeheader()
        writer.writerows(data_list)
    print(f"✅ Saved {len(data_list)} rows → {filename}")


In [ ]:
#req again with new link from matchlinks
import re

#fight_link=[]
# define before your for-loop- to save 
round1_data, round2_data, round3_data = [], [], [] #save same fight that end with same total round in 1 file

fight_rows = []
for match in match_links:
    response = requests.get(match)
    soup = BeautifulSoup(response.text, "html.parser")

    #in each match link table
    #find all right row
    fight_row = soup.find_all('tr', class_="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click")
    fight_rows.append(fight_row)

    for row in fight_rows:
        tds = row.find_all("td") #find all td in each fight row
        if len(tds) >= 9: #total row is 9 - 0~9
            #get data in each row
            #get weight in 6 row. find the p tag with info
            weight_class = tds[6].find("p").get_text(strip=True)
            #get method in 7 row. have 2 p tag get the first one
            method = tds[7].find_all("p")[0].get_text(strip=True)
            #get total round in 8 row
            total_round = tds[8].find("p").get_text(strip=True) #1,2,3 - total round that ends

            #get only matching category
            category = weight_class == "Lightweight" and (method in ["KO/TKO", "KO", "TKO"])
            if category:
                #print("match found",weight_class, method)
                #get data link to go to the stats table and extract data 
                data_link = row.get("data-link") # all fight link that matched the category
                #print(data_link)

                # req again with the fight link to go to stats table
                data_link_response = requests.get(data_link)
                soup = BeautifulSoup(data_link_response.text, "html.parser")

                # extract data in stats table page
                fight_details = soup.find("div", class_="b-fight-details__fight")
                if fight_details:
                    # get round num
                    total_round = fight_details.find("i", class_="b-fight-details__text-item").get_text(strip=True)
                    # get method ko/tko
                    method_tag = fight_details.find("i", attrs={"style": "font-style: normal"})
                    method = method_tag.get_text(strip=True) if method_tag else "-"

                # extract data from table
                table_stats = soup.find_all("table", class_="b-fight-details__table js-fight-table")
                #print(len(table_stats)) #2
                table1= table_stats[0] #top totals table third table
                table2= table_stats[1] #significant table body attack forth table

                                                    

In [ ]:
#req again with new link from matchlinks
import re

fight_link=[]
# define before your for-loop- to save 
round1_data, round2_data, round3_data = [], [], [] #save same fight that end with same total round in 1 file

for match in match_links:
    response = requests.get(match)
    soup = BeautifulSoup(response.text, "html.parser")

    #in each match link table
    #find all right row
    fight_rows = soup.find_all('tr', class_="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click")

    for row in fight_rows:
        tds = row.find_all("td") #find all td in each fight row
        if len(tds) >= 9: #total row is 9 - 0~9
            #get data in each row
            #get weight in 6 row. find the p tag with info
            weight_class = tds[6].find("p").get_text(strip=True)
            #get method in 7 row. have 2 p tag get the first one
            method = tds[7].find_all("p")[0].get_text(strip=True)
            #get total round in 8 row
            total_round = tds[8].find("p").get_text(strip=True) #1,2,3 - total round that ends

            #get only matching category
            category = weight_class == "Lightweight" and (method in ["KO/TKO", "KO", "TKO"])
            if category:
                #print("match found",weight_class, method)
                #get data link to go to the stats table and extract data 
                data_link = row.get("data-link") # all fight link that matched the category
                #print(data_link)

                # req again with the fight link to go to stats table
                data_link_response = requests.get(data_link)
                soup = BeautifulSoup(data_link_response.text, "html.parser")

                # extract data in stats table page
                fight_details = soup.find("div", class_="b-fight-details__fight")
                if fight_details:
                    # get round num
                    total_round = fight_details.find("i", class_="b-fight-details__text-item").get_text(strip=True)
                    # get method ko/tko
                    method_tag = fight_details.find("i", attrs={"style": "font-style: normal"})
                    method = method_tag.get_text(strip=True) if method_tag else "-"

                # extract data from table
                table_stats = soup.find_all("table", class_="b-fight-details__table js-fight-table")
                #print(len(table_stats)) #2
                table1= table_stats[0] #top totals table third table
                table2= table_stats[1] #significant table body attack forth table
                 #extract data for one match from 2 table. 
                #now need to do this for all match 

                if table_stats:
                    for section in table1.find_all("tr", class_="b-fight-details__table-row"):
                        td_stats = section.find_all("td")
                        if len(td_stats) >= 10:  # total col is 10
                            # get data in each row ex: round 1
                            kd = [p.get_text(strip=True) for p in td_stats[1].find_all("p")]
                            sig_str = [p.get_text(strip=True) for p in td_stats[2].find_all("p")]
                            sig_str_pct = [p.get_text(strip=True) for p in td_stats[3].find_all("p")]
                            total_str = [p.get_text(strip=True) for p in td_stats[4].find_all("p")]
                            td_attempt = [p.get_text(strip=True) for p in td_stats[5].find_all("p")]
                            td_pct = [p.get_text(strip=True) for p in td_stats[6].find_all("p")]
                            sub_att = [p.get_text(strip=True) for p in td_stats[7].find_all("p")]
                            rev = [p.get_text(strip=True) for p in td_stats[8].find_all("p")]
                            ctrl_sec = [p.get_text(strip=True) for p in td_stats[9].find_all("p")]
                            # each list above has [fighter1, fighter2]
                            
                    for section2 in table2.find_all("tr", class_="b-fight-details__table-row"):
                        td_stats_2 = section2.find_all("td")
                        if len(td_stats_2) >= 8:
                            #get body attack data
                            head = [p.get_text(strip=True) for p in td_stats_2[3].find_all("p")]
                            body = [p.get_text(strip=True) for p in td_stats_2[4].find_all("p")]
                            leg = [p.get_text(strip=True) for p in td_stats_2[5].find_all("p")]
                            distance = [p.get_text(strip=True) for p in td_stats_2[6].find_all("p")]
                            clinch = [p.get_text(strip=True) for p in td_stats_2[7].find_all("p")]
                            ground = [p.get_text(strip=True) for p in td_stats_2[8].find_all("p")]

                            print("KD:", kd,
                                "Sig.Str (L/A):", sig_str,
                                "sig_str_pct:", sig_str_pct,
                                "total_str (L/A):", total_str,
                                "td (L/A):", td_attempt,
                                "td_pct:", td_pct,
                                "sub_att:", sub_att,
                                "rev:", rev,
                                "Control(sec):", ctrl_sec,
                                "head (L/A):", head,
                                "body (L/A):", body,
                                "leg (L/A):", leg,
                                "distance (L/A):", distance,
                                "clinch (L/A):", clinch,
                                "ground (L/A):", ground,
                                "total_round:", total_round,
                                )

In [6]:


#extract data for one match from 2 table. 
#now need to do this for all match 

if table_stats:
    for section in table1.find_all("tr", class_="b-fight-details__table-row"):
        td_stats = section.find_all("td")
        if len(td_stats) >= 10:  # total col is 10
            # get data in each row ex: round 1
            kd = [p.get_text(strip=True) for p in td_stats[1].find_all("p")]
            sig_str = [p.get_text(strip=True) for p in td_stats[2].find_all("p")]
            sig_str_pct = [p.get_text(strip=True) for p in td_stats[3].find_all("p")]
            total_str = [p.get_text(strip=True) for p in td_stats[4].find_all("p")]
            td_attempt = [p.get_text(strip=True) for p in td_stats[5].find_all("p")]
            td_pct = [p.get_text(strip=True) for p in td_stats[6].find_all("p")]
            sub_att = [p.get_text(strip=True) for p in td_stats[7].find_all("p")]
            rev = [p.get_text(strip=True) for p in td_stats[8].find_all("p")]
            ctrl_sec = [p.get_text(strip=True) for p in td_stats[9].find_all("p")]
            # each list above has [fighter1, fighter2]
            
    for section2 in table2.find_all("tr", class_="b-fight-details__table-row"):
        td_stats_2 = section2.find_all("td")
        if len(td_stats_2) >= 8:
             #get body attack data
            head = [p.get_text(strip=True) for p in td_stats_2[3].find_all("p")]
            body = [p.get_text(strip=True) for p in td_stats_2[4].find_all("p")]
            leg = [p.get_text(strip=True) for p in td_stats_2[5].find_all("p")]
            distance = [p.get_text(strip=True) for p in td_stats_2[6].find_all("p")]
            clinch = [p.get_text(strip=True) for p in td_stats_2[7].find_all("p")]
            ground = [p.get_text(strip=True) for p in td_stats_2[8].find_all("p")]

            print("KD:", kd,
                "Sig.Str (L/A):", sig_str,
                "sig_str_pct:", sig_str_pct,
                "total_str (L/A):", total_str,
                "td (L/A):", td_attempt,
                "td_pct:", td_pct,
                "sub_att:", sub_att,
                "rev:", rev,
                "Control(sec):", ctrl_sec,
                "head (L/A):", head,
                "body (L/A):", body,
                "leg (L/A):", leg,
                "distance (L/A):", distance,
                "clinch (L/A):", clinch,
                "ground (L/A):", ground,
                "total_round:", total_round,
                )

KD: ['0', '0'] Sig.Str (L/A): ['12 of 25', '1 of 4'] sig_str_pct: ['48%', '25%'] total_str (L/A): ['15 of 28', '1 of 4'] td (L/A): ['1 of 1', '0 of 0'] td_pct: ['100%', '---'] sub_att: ['0', '0'] rev: ['0', '0'] Control(sec): ['0:50', '0:00'] head (L/A): ['11 of 28', '3 of 11'] body (L/A): ['5 of 8', '1 of 1'] leg (L/A): ['6 of 7', '4 of 4'] distance (L/A): ['6 of 17', '8 of 15'] clinch (L/A): ['0 of 0', '0 of 1'] ground (L/A): ['16 of 26', '0 of 0'] total_round: 3
KD: ['0', '0'] Sig.Str (L/A): ['12 of 25', '1 of 4'] sig_str_pct: ['48%', '25%'] total_str (L/A): ['15 of 28', '1 of 4'] td (L/A): ['1 of 1', '0 of 0'] td_pct: ['100%', '---'] sub_att: ['0', '0'] rev: ['0', '0'] Control(sec): ['0:50', '0:00'] head (L/A): ['20 of 38', '3 of 13'] body (L/A): ['5 of 5', '2 of 4'] leg (L/A): ['0 of 0', '2 of 2'] distance (L/A): ['9 of 21', '5 of 17'] clinch (L/A): ['0 of 0', '1 of 1'] ground (L/A): ['16 of 22', '1 of 1'] total_round: 3
KD: ['0', '0'] Sig.Str (L/A): ['12 of 25', '1 of 4'] sig_str

In [9]:
if table_stats:
                 # get each round num ex: Round 1, Round 2, Round 3
                    #table1 = [th.get_text(strip=True) for th in table_stats.find_all("th", colspan="10")]
                    #current_round = None
                    for section in table1.find_all("thead", class_="b-fight-details__table-row b-fight-details__table-row_type_head"):
                            # check if this row is a round header
                            header_th = section.find("th", class_="b-fight-details__table-col")
                            if header_th:
                                current_round = header_th.get_text(strip=True)  # ex: "Round 1"

                            # iterate over each fight row
                            for r in table1.find_all("tr", class_="b-fight-details__table-row"):
                                td_stats = r.find_all("td")
                                if len(td_stats) >= 10:  # total col is 10
                                    # get data in each row ex: round 1
                                    kd = [p.get_text(strip=True) for p in td_stats[1].find_all("p")]
                                    sig_str = [p.get_text(strip=True) for p in td_stats[2].find_all("p")]
                                    sig_str_pct = [p.get_text(strip=True) for p in td_stats[3].find_all("p")]
                                    total_str = [p.get_text(strip=True) for p in td_stats[4].find_all("p")]
                                    td_attempt = [p.get_text(strip=True) for p in td_stats[5].find_all("p")]
                                    td_pct = [p.get_text(strip=True) for p in td_stats[6].find_all("p")]
                                    sub_att = [p.get_text(strip=True) for p in td_stats[7].find_all("p")]
                                    rev = [p.get_text(strip=True) for p in td_stats[8].find_all("p")]
                                    ctrl_sec = [p.get_text(strip=True) for p in td_stats[9].find_all("p")]
                                    # each list above has [fighter1, fighter2]
                 
                            # APPLY CLEANING
                            sig_str = split_of_list(sig_str)
                            total_str = split_of_list(total_str)
                            td_attempt = split_of_list(td_attempt)
                            sig_str_pct = clean_pct_list(sig_str_pct)
                            td_pct = clean_pct_list(td_pct)
                            sub_att = clean_int_list(sub_att)
                            rev = clean_int_list(rev)
                            kd = clean_int_list(kd)
                            ctrl_sec = ctrl_to_seconds(ctrl_sec)

                            #print("round:", current_round,
                             #   "KD:", kd,
                              #  "Sig.Str (L/A):", sig_str,
                               # "sig_str_pct:", sig_str_pct,
                                #"total_str (L/A):", total_str,
                                #"td (L/A):", td_attempt,
                                #"td_pct:", td_pct,
                                #"sub_att:", sub_att,
                                #"rev:", rev,
                                #"Control(sec):", ctrl_sec)


                            # iterate over all rows in table
                            # get each round num ex: Round 1, Round 2, Round 3                 
                            #current_round_2 = None
                    for section in table2.find_all("thead", class_="b-fight-details__table-row b-fight-details__table-row_type_head"):
                            # check if this row is a round header
                            header_th = section.find("th", class_="b-fight-details__table-col")
                            if header_th:
                                current_round_2 = header_th.get_text(strip=True)  # e.g., "Round 1"

                            #tbody = section.find_next("tbody") #avoid double loop from header
                            #if not tbody:
                                #continue

                            for rr in table2.find_all("tr", class_="b-fight-details__table-row"):
                                td_stats_2 = rr.find_all("td")                    
                                if len(td_stats_2) >= 8:
                                    #get body attack data
                                    head = [p.get_text(strip=True) for p in td_stats_2[3].find_all("p")]
                                    body = [p.get_text(strip=True) for p in td_stats_2[4].find_all("p")]
                                    leg = [p.get_text(strip=True) for p in td_stats_2[5].find_all("p")]
                                    distance = [p.get_text(strip=True) for p in td_stats_2[6].find_all("p")]
                                    clinch = [p.get_text(strip=True) for p in td_stats_2[7].find_all("p")]
                                    ground = [p.get_text(strip=True) for p in td_stats_2[8].find_all("p")]

                                    # each list above has [fighter1, fighter2]
                                  # clean "of" patterns same as before
                            head = split_of_list(head)
                            body = split_of_list(body)
                            leg = split_of_list(leg)
                            distance = split_of_list(distance)
                            clinch = split_of_list(clinch)
                            ground = split_of_list(ground)

                            #print("round:", current_round_2,
                               # "head (L/A):", head,
                               # "body (L/A):", body,
                               # "leg (L/A):", leg,
                                #"distance (L/A):", distance,
                                #"clinch (L/A):", clinch,
                                #"ground (L/A):", ground,
                                #"total_round:", total_round,
                                #"win method:", method
                                #)
                            
                            data = {
                            "Round": current_round_2,
                            "KD": kd,
                            "Sig_Str": sig_str,
                            "Sig_Str_%": sig_str_pct,
                            "Total_Str": total_str,
                            "Td": td_attempt,
                            "Td_%": td_pct,
                            "Sub_Att": sub_att,
                            "Rev": rev,
                            "Ctrl(sec)": ctrl_sec,
                            "Head(L/A)": head,
                            "Body(L/A)": body,
                            "Leg(L/A)": leg,
                            "Distance(L/A)": distance,
                            "Clinch(L/A)": clinch,
                            "Ground(L/A)": ground,
                            "Total_Round": total_round,
                            "Win_Method": method
                        }

                            if total_round == "Round:1":
                                round1_data.append(data)
                            elif total_round == "Round:2":
                                round2_data.append(data)
                            elif total_round == "Round:3":
                                round3_data.append(data)

save_to_csv("round1_1.csv", round1_data)
save_to_csv("round2_2.csv", round2_data)
save_to_csv("round3_3.csv", round3_data)

⚠️ No data to save for round1_1.csv
⚠️ No data to save for round2_2.csv
⚠️ No data to save for round3_3.csv


In [6]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# === Start main crawl ===

round1_data, round2_data, round3_data = [], [], []
fight_id = 0

for match in match_links:
    response = requests.get(match)
    soup = BeautifulSoup(response.text, "html.parser")

    fight_rows = soup.find_all(
        'tr',
        class_="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click"
    )

    for row in fight_rows:
        tds = row.find_all("td")
        if len(tds) >= 9:
            weight_class = tds[6].find("p").get_text(strip=True)
            method = tds[7].find_all("p")[0].get_text(strip=True)
            total_round = tds[8].find("p").get_text(strip=True)

            if weight_class == "Lightweight" and method in ["KO/TKO", "KO", "TKO"]:
                data_link = row.get("data-link")
                data_link_response = requests.get(data_link)
                soup = BeautifulSoup(data_link_response.text, "html.parser")

                fight_details = soup.find("div", class_="b-fight-details__fight")
                if fight_details:
                    total_round = fight_details.find("i", class_="b-fight-details__text-item").get_text(strip=True)
                    method_tag = fight_details.find("i", attrs={"style": "font-style: normal"})
                    method = method_tag.get_text(strip=True) if method_tag else "-"

                table_stats = soup.find_all("table", class_="b-fight-details__table js-fight-table")
                if not table_stats or len(table_stats) < 2:
                    continue

                table1, table2 = table_stats
                round_data = []  # store all rounds for this fight

                # Extract headers
                headers = table1.find_all("th", class_="b-fight-details__table-col")
                round_headers = [th.get_text(strip=True) for th in headers if "Round" in th.get_text()]

                for current_round in round_headers:
                    # Extract table1 (totals)
                    for r in table1.find_all("tr", class_="b-fight-details__table-row"):
                        td_stats = r.find_all("td")
                        if len(td_stats) >= 10:
                            kd = [p.get_text(strip=True) for p in td_stats[1].find_all("p")]
                            sig_str = [p.get_text(strip=True) for p in td_stats[2].find_all("p")]
                            sig_str_pct = [p.get_text(strip=True) for p in td_stats[3].find_all("p")]
                            total_str = [p.get_text(strip=True) for p in td_stats[4].find_all("p")]
                            td_attempt = [p.get_text(strip=True) for p in td_stats[5].find_all("p")]
                            td_pct = [p.get_text(strip=True) for p in td_stats[6].find_all("p")]
                            sub_att = [p.get_text(strip=True) for p in td_stats[7].find_all("p")]
                            rev = [p.get_text(strip=True) for p in td_stats[8].find_all("p")]
                            ctrl_sec = [p.get_text(strip=True) for p in td_stats[9].find_all("p")]

                            # Clean with regex
                            sig_str = split_of_list(sig_str)
                            total_str = split_of_list(total_str)
                            td_attempt = split_of_list(td_attempt)
                            sig_str_pct = clean_pct_list(sig_str_pct)
                            td_pct = clean_pct_list(td_pct)
                            sub_att = clean_int_list(sub_att)
                            rev = clean_int_list(rev)
                            kd = clean_int_list(kd)
                            ctrl_sec = ctrl_to_seconds(ctrl_sec)

                    # Extract table2 (significant strikes)
                    for rr in table2.find_all("tr", class_="b-fight-details__table-row"):
                        td_stats_2 = rr.find_all("td")
                        if len(td_stats_2) >= 8:
                            head = [p.get_text(strip=True) for p in td_stats_2[3].find_all("p")]
                            body = [p.get_text(strip=True) for p in td_stats_2[4].find_all("p")]
                            leg = [p.get_text(strip=True) for p in td_stats_2[5].find_all("p")]
                            distance = [p.get_text(strip=True) for p in td_stats_2[6].find_all("p")]
                            clinch = [p.get_text(strip=True) for p in td_stats_2[7].find_all("p")]
                            ground = [p.get_text(strip=True) for p in td_stats_2[8].find_all("p")]

                            head = split_of_list(head)
                            body = split_of_list(body)
                            leg = split_of_list(leg)
                            distance = split_of_list(distance)
                            clinch = split_of_list(clinch)
                            ground = split_of_list(ground)

                    # Store combined round data
                    data = {
                        "Fight_ID": fight_id,
                        "Round": current_round,
                        "KD": kd,
                        "Sig_Str": sig_str,
                        "Sig_Str_%": sig_str_pct,
                        "Total_Str": total_str,
                        "Td": td_attempt,
                        "Td_%": td_pct,
                        "Sub_Att": sub_att,
                        "Rev": rev,
                        "Ctrl(sec)": ctrl_sec,
                        "Head(L/A)": head,
                        "Body(L/A)": body,
                        "Leg(L/A)": leg,
                        "Distance(L/A)": distance,
                        "Clinch(L/A)": clinch,
                        "Ground(L/A)": ground,
                        "Total_Round": total_round,
                        "Win_Method": method,
                    }
                    round_data.append(data)

                # Append to correct file (based on ending round)
                if total_round == "1" or "Round:1" in total_round:
                    round1_data.extend(round_data)
                elif total_round == "2" or "Round:2" in total_round:
                    round2_data.extend(round_data)
                elif total_round == "3" or "Round:3" in total_round:
                    round3_data.extend(round_data)

                fight_id += 1
                print(f"✅ Fight {fight_id} saved ({method}, ended {total_round})")

# Save all
save_to_csv("round1.csv", round1_data)
save_to_csv("round2.csv", round2_data)
save_to_csv("round3.csv", round3_data)


✅ Fight 1 saved (KO/TKO, ended Round:1)
✅ Fight 2 saved (KO/TKO, ended Round:3)
✅ Fight 3 saved (KO/TKO, ended Round:3)
✅ Fight 4 saved (KO/TKO, ended Round:2)
✅ Fight 5 saved (KO/TKO, ended Round:2)
✅ Fight 6 saved (KO/TKO, ended Round:1)
✅ Fight 7 saved (KO/TKO, ended Round:1)
✅ Fight 8 saved (KO/TKO, ended Round:2)
✅ Fight 9 saved (KO/TKO, ended Round:1)
✅ Fight 10 saved (KO/TKO, ended Round:2)
✅ Fight 11 saved (KO/TKO, ended Round:1)
✅ Fight 12 saved (KO/TKO, ended Round:3)
✅ Fight 13 saved (KO/TKO, ended Round:1)
✅ Fight 14 saved (KO/TKO, ended Round:1)
✅ Fight 15 saved (TKO - Doctor's Stoppage, ended Round:1)
✅ Fight 16 saved (KO/TKO, ended Round:1)
✅ Fight 17 saved (KO/TKO, ended Round:1)
✅ Fight 18 saved (KO/TKO, ended Round:2)
✅ Fight 19 saved (KO/TKO, ended Round:1)
✅ Fight 20 saved (KO/TKO, ended Round:2)
✅ Fight 21 saved (TKO - Doctor's Stoppage, ended Round:2)
✅ Fight 22 saved (KO/TKO, ended Round:3)
✅ Fight 23 saved (KO/TKO, ended Round:1)
✅ Fight 24 saved (TKO - Doctor's

In [ ]:
#if fight A total round end at 3
#extract all round data in that link in the data stat table and
#all round mean in fight A's round1,round2,round3.
#save it to round3 file

#try with one fight first
#let say fight
#http://www.ufcstats.com/fight-details/7cbfeba85f86d1bf
#in here the total round is 3
#so in the stats table have data for each round 1,2 and 3
# extract the data in each round in table totals for KD	Sig. str.	Sig. str. %	Total str.	Td	Td %	Sub. att	Rev.	Ctrl and
# the data in significant strikes table of Head	Body	Leg	Distance	Clinch	Ground
#we dont need the Sig. str	Sig. str. % again since duplicate
#save the extract data in round3 file.

In [3]:
#try use id if keep repeated, coz id is uniques so will nop repeat

In [ ]:
 #req again with the fight link to go to stats table
                data_link_response = requests.get(data_link)
                soup = BeautifulSoup(data_link_response.text, "html.parser")
                
                #extract data in stats table page
                #get total_round first on top of table
                fight_details = soup.find("div", class_="b-fight-details__fight")
                if fight_details:
                    total_round = fight_details.find("i", class_="b-fight-details__text-item").get_text(strip=True) #get round num
                    method = fight_details.find("i", style_="font-style: normal").get_text(strip=True) # get method

                #extract data from table
                table_stats = soup.find("table", class_="b-fight-details__table js-fight-table")
                if table_stats:
                    round_num = table_stats.find.all("th", class_="b-fight-details__table-col").get_text(strip=True)#get each round num ex:round1

                table_stats_row = soup.find("tr", class_="b-fight-details__table-row")
                for r in table_stats_row:
                    td_stats = row.find_all("td")
                    if len(td_stats)>=9:
                        #get data in each row ex: round 1
                        #fight_id =  [i++]
                        kd = td_stats[1].find("p").get_text(strip=True) #0
                        sig_str_landed = td_stats[2].find("p")[1].get_text(strip=True) #22
                        sig_str_attempted = td_stats[2].find("p")[2].get_text(strip=True) #41
                        sid_str_p = td_stats[3].find("p").get_text(strip=True) #53
                        total_str_landed = td_stats[4].find("p")[1].get_text(strip=True) #23
                        total_str_attempted = td_stats[4].find("p")[2].get_text(strip=True),#42
                        td_landed = td_stats[5].find("p")[1].get_text(strip=True) #1
                        td_attempted = td_stats[5].find("p")[2].get_text(strip=True) #1
                        td_p = td_stats[6].find("p").get_text(strip=True) #100
                        sub_att = td_stats[7].find("p").get_text(strip=True) #0
                        rev =  td_stats[8].find("p").get_text(strip=True) #0
                        ctrl_sec = td_stats[9].find("p").get_text(strip=True) #24 - convert to second
